In [1]:
import numpy as np

In [4]:
import pyphi

![title](phi_algorithm.png)

In [144]:
class phi():
    def __init__(self,tpm):
        self.tpm = tpm
        self.num_nodes = int(np.log2(len(tpm)))
        
    # Converts a list of states to a row number, so that we can index the tpm
    # input : list of states, Ex. [0,1,0,0]
    # output : row number based on little endian notation, so index zero in list is least
    #          significant bit and index n-1 is most significant bit. Ex. [0,1,0,0] -> 2
    def state_to_index(self,states):
        decimal = 0
        for i,state in enumerate(states):
            decimal += (2**i) * (state)
        return decimal
    
    # Converts a row number from the tpm into a list of states
    # input : row number, total number of states. Ex. (2, 4)
    # output : list of states that row number represents. Ex. (2, 4) -> [0,1,0,0]
    #          We include the number of states so that we know how many zeros to append at the end.
    def index_to_state(self,index,num_states):
        binary = bin(index)[2:]
        states = []
        for state in binary: # convert row to binary
            states.append(int(state))
        
        states = np.flip(states) # flip to make little endian
       
        if (len(states) < num_states): 
            states = np.concatenate((states,np.zeros((num_states-len(states)),dtype=int)))
            
        return states
    
    # Because they're conditional probability distributions, the rows need to sum to one
    def normalize_rows(self,tpm):
        return tpm/np.sum(tpm,1)[:, np.newaxis]
    
    # return the powerset of a list of nodes
    def powerset(self,iterable):
        "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
        s = list(iterable)
        return chain.from_iterable(combinations(s, r) for r in range(1,len(s)+1))
    
    # returns the tensor product of two tpm (effect)
    def tensor_product(self,t1,t2):
        tensor_product = np.zeros((t1.shape[0],t1.shape[1]*t2.shape[1]))
        row = 0
        for c2 in t2.T:
            for c1 in t1.T:
                tensor_product[:,row] = c1*c2
                row+=1
                
        return tensor_product
        
    """
    effect_repertoire :
        input : mechanism (t nodes), purview (t+1 nodes)
        algorithm : from original tpm find the tpm P(purview | mechanism)
        output : repertoire for those nodes
    """    
    def effect_repertoire(self,mechanism,purview):
        # The effect repertoire captures the conditional transition probability of transitioning to each
        # purview state, given the current mechanism state, so it needs to be of size
        # NUMBER POSSIBLE MECHANISM STATES X NUMBER POSSIBLE PURVIEW STATES, but we start by marginalizing
        # over only the mechanism states, so NUMBER POSSIBLE MECHANISM STATES X NUMBER POSSIBLE STATES
        mechanism_effect_repertoire = np.zeros((2**len(mechanism),2**self.num_nodes))
        
        # We marginalize over the mechanism states, which means we sum the probabilities of rows which
        # only differ in the state of nodes not in the mechanism.
        # We do this by finding the mechanism's state for a given row and mapping that row in the original tpm
        # to the correct row in the new mechanism repertoire
        for row in range(self.tpm.shape[0]):
            mechanism_state = self.index_to_state(row,self.num_nodes)[mechanism]
            mechanism_effect_repertoire[self.state_to_index(mechanism_state),:] += self.tpm[row,:]
        
        # This is the final effect repertoire
        effect_repertoire = np.zeros((2**len(mechanism),2**len(purview)))
        
        # Second, we marginalize over the column states, which means we sum the probabilities of columns which
        # only differ in the state of nodes not in the purview.
        # We do this by finding the purview's state for a given column and mapping that row in the original tpm
        # to the correct column in the new effect repertoire
        for column in range(self.tpm.shape[1]):
            purview_state = self.index_to_state(column,self.num_nodes)[purview]
            effect_repertoire[:,self.state_to_index(purview_state)] += mechanism_effect_repertoire[:,column]
        
        # All that's left to do is normalize the rows because each row is a conditional probability distribution
        effect_repertoire = self.normalize_rows(effect_repertoire)
        
        # Now, we have to expand the effect_repertoire into the original state space which has all the 
        # possible current states at time t
        expanded_effect_repertoire = np.zeros((2**self.num_nodes,2**len(purview)))
        
        # This is done by mapping distributions in the effect_repertoire to each row in the expanded repertoire
        # where the mechanism's state matches
        for row in range(2**self.num_nodes):
            mechanism_state = self.index_to_state(row,self.num_nodes)[mechanism]
            expanded_effect_repertoire[row,:] = effect_repertoire[self.state_to_index(mechanism_state),:]
            
        return expanded_effect_repertoire
    
    def effect_mip(self,system,purview):
        pass
        """
        for each possible partition of the pair (system + purview):
            for each partitioned subset:
                partitioned tpm = tensor multiply partitioned subset tpms 
                                    (effect_repertoire(part_system,part_purview))
            calculate distance between partitioned tpm and effect_repertoire(system,purview)
        return smallest distance
        """
        
    
    """
    effect_mip : 
        input : system (t nodes) , purview (t+1 nodes)
        
        algorithm : for each possible partition of the pair (system + purview):
                        for each partitioned subset:
                            partitioned tpm = tensor multiply partitioned subset tpms 
                                                (effect_repertoire(part_system,part_purview))
                        calculate distance between partitioned tpm and effect_repertoire(system,purview)
                    return smallest distance
                    
        output : phi (distance between effect repertoire and minimum information partition repertoire)
        
    mie : 
        input : tpm for a specific system (in our case this is the entire system)
        
        algorithm : for each subset of nodes at t+1 of the system (call this a purview):
                        call effect_mip(system, purview)
                    return max effect_mip
        
        output : effect phi value
                        
        
    concept :
        input : list of nodes in system (in our case this will be the whole system)
        
        algorithm : min (mic(system),mie(system))
        
        output : phi value for that system
    """

In [145]:
test = phi(np.array([[1., 0., 0., 0., 0., 0., 0., 0.],
                   [0., 0., 0., 0., 1., 0., 0., 0.],
                   [0., 0., 0., 0., 0., 1., 0., 0.],
                   [0., 1., 0., 0., 0., 0., 0., 0.],
                   [0., 1., 0., 0., 0., 0., 0., 0.],
                   [0., 0., 0., 0., 0., 0., 0., 1.],
                   [0., 0., 0., 0., 0., 1., 0., 0.],
                   [0., 0., 0., 1., 0., 0., 0., 0.]], dtype=int))

In [146]:
test.tpm

array([[1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0]])

In [147]:
p1 = test.tensor_product(test.effect_repertoire([1,2],[0]),test.effect_repertoire([2],[1]))
print (p1)
test.tensor_product(p1,test.effect_repertoire([1],[2]))

[[1.  0.  0.  0. ]
 [1.  0.  0.  0. ]
 [0.  1.  0.  0. ]
 [0.  1.  0.  0. ]
 [0.  0.5 0.  0.5]
 [0.  0.5 0.  0.5]
 [0.  0.5 0.  0.5]
 [0.  0.5 0.  0.5]]


array([[0.5 , 0.  , 0.  , 0.  , 0.5 , 0.  , 0.  , 0.  ],
       [0.5 , 0.  , 0.  , 0.  , 0.5 , 0.  , 0.  , 0.  ],
       [0.  , 0.5 , 0.  , 0.  , 0.  , 0.5 , 0.  , 0.  ],
       [0.  , 0.5 , 0.  , 0.  , 0.  , 0.5 , 0.  , 0.  ],
       [0.  , 0.25, 0.  , 0.25, 0.  , 0.25, 0.  , 0.25],
       [0.  , 0.25, 0.  , 0.25, 0.  , 0.25, 0.  , 0.25],
       [0.  , 0.25, 0.  , 0.25, 0.  , 0.25, 0.  , 0.25],
       [0.  , 0.25, 0.  , 0.25, 0.  , 0.25, 0.  , 0.25]])